In [ ]:
import pandas as pd

In [76]:
metadata_df = pd.read_csv("../../DafnyBench/metadata/ground_truth_metadata.csv")
metadata_df

,test_ID,test_file,has_requires,has_ensures,#_char,#_hint_char,#_lemma,#_function,#_method,verifies_without_hints
0,0,630-dafny_tmp_tmpz2kokaiq_Solution.dfy,True,True,781,89,0,1,1,False
1,1,703FinalProject_tmp_tmpr_10rn4z_DP-GD.dfy,True,False,1452,231,0,0,1,True
2,2,703FinalProject_tmp_tmpr_10rn4z_gaussian.dfy,True,False,1028,274,0,1,1,True
3,3,AssertivePrograming_tmp_tmpwf43uz0e_DivMode_Un...,True,True,9500,1745,14,7,3,False
4,4,AssertivePrograming_tmp_tmpwf43uz0e_Find_Subst...,True,True,10254,3595,5,0,2,False
...,...,...,...,...,...,...,...,...,...,...
777,777,verified-using-dafny_tmp_tmp7jatpjyn_longestZe...,True,True,2274,652,0,1,2,False
778,778,vfag_tmp_tmpc29dxm1j_Verificacion_torneo.dfy,True,True,8315,6019,0,0,1,True
779,779,vfag_tmp_tmpc29dxm1j_mergesort.dfy,True,False,2528,239,0,0,3,False
780,780,vfag_tmp_tmpc29dxm1j_sumar_componentes.dfy,True,True,1507,779,0,1,1,False


In [77]:
claude4_opus_results_df = pd.read_csv("../results_summary/claude-opus-4.1_results.csv")
gpt5_results_df = pd.read_csv("../results_summary/gpt-5_results.csv")
gpt_5_mini_results_df = pd.read_csv("../results_summary/gpt-5-mini_results.csv")
grok_results_df = pd.read_csv("../results_summary/grok-code-fast-1_results.csv")
claude4_sonnet_results_df = pd.read_csv("../results_summary/claude-sonnet-4_results.csv")
gemini_2_5__pro_results_df = pd.read_csv("../results_summary/gemini-2.5-pro_results.csv")
glm_4_5_results_df = pd.read_csv("../results_summary/glm-4.5_results.csv")
deepseek_chat_3_1_results_df = pd.read_csv("../results_summary/deepseek-chat-v3.1_results.csv")
gemini_2_5_flash_results_df = pd.read_csv("../results_summary/gemini-2.5-flash_results.csv")
dfs = {
    'claude-opus-4.1': claude4_opus_results_df,
    'gpt-5': gpt5_results_df,
    'grok-code-fast-1': grok_results_df,
    'claude-sonnet-4': claude4_sonnet_results_df,
    'gemini-2.5-pro': gemini_2_5__pro_results_df,
    'glm-4.5': glm_4_5_results_df,
    'deepseek-chat-v3.1': deepseek_chat_3_1_results_df,
    'gemini-2.5-flash': gemini_2_5_flash_results_df,
    'gpt-5-mini': gpt_5_mini_results_df
}

for model, df in dfs.items():
    # df = df[df['success_on_attempt_#'] != 'failed api']
    df['success_on_attempt_#'] = df['success_on_attempt_#'].replace('failed api', 'failed')
    # Only keep unique test_IDs for each model
    # Sort by success status (failed/failed api last) then drop duplicates, keeping first (non-failed if exists)
    df['sort_key'] = df['success_on_attempt_#'].apply(lambda x: 0 if x not in ['failed', 'failed api'] else 1)
    df = df.sort_values(['test_ID', 'sort_key']).drop_duplicates(subset=['test_ID']).drop(columns=['sort_key'])

    dfs[model] = df

In [78]:
for model_name, df in dfs.items():
    total_success_rate = 1 - (len(df[df['success_on_attempt_#'] == 'failed']) / len(df))
    print(f"Total success rate for {model_name}: {total_success_rate} on {len(df)} files")

Total success rate for claude-opus-4.1: 0.8887468030690537 on 782 files
Total success rate for gpt-5: 0.7659846547314578 on 782 files
Total success rate for grok-code-fast-1: 0.8209718670076727 on 782 files
Total success rate for claude-sonnet-4: 0.8631713554987213 on 782 files
Total success rate for gemini-2.5-pro: 0.7161125319693095 on 782 files
Total success rate for glm-4.5: 0.7621483375959079 on 782 files
Total success rate for deepseek-chat-v3.1: 0.7289002557544757 on 782 files
Total success rate for gemini-2.5-flash: 0.6624040920716112 on 782 files
Total success rate for gpt-5-mini: 0.870843989769821 on 782 files


In [79]:
# Get union of all test IDs across all models
all_test_ids = set()
all_test_files = set()
for df in dfs.values():
    all_test_ids.update(df['test_ID'])
    all_test_files.update(df['test_file'])

print(f"Total unique test IDs across all models: {len(all_test_ids)}")

# Create union results - a test succeeds if ANY model succeeds on it
union_results = []
for test_id in all_test_ids:
    succeeded = False
    for model_name, df in dfs.items():
        test_row = df[df['test_ID'] == test_id]
        if not test_row.empty:
            if test_row.iloc[0]['success_on_attempt_#'] != 'failed' and test_row.iloc[0]['success_on_attempt_#'] != 'failed api':
                succeeded = True
                break
    union_results.append({'test_ID': test_id, 'union_success': succeeded})

union_df = pd.DataFrame(union_results)
union_success_rate = union_df['union_success'].mean()
print(f"Union success rate (any model succeeds): {union_success_rate:.4f} on {len(union_df)} files")


Total unique test IDs across all models: 782
Union success rate (any model succeeds): 0.9731 on 782 files


Plot the model union progress over time